In [ ]:
import itertools
import os
import graphviz

import pandas
import pandas as pd

import sqlite3

In [ ]:
kc1 = "alphad3m-ds_splice-seed_662873"
temp =  "temp"
sqlite_db =  "db.sqlite3"
task_grammar = "task_grammar.bnf"



In [ ]:
def connect_to_sqlite(path: str) -> (sqlite3.Connection, sqlite3.Cursor):
    """
    connects and returns connections and cursor for the sqlight3 database from the path
    :param path: str full path of the sql db
    :return: connection, cursor
    """

    if os.path.exists(path):

        conn = sqlite3.connect(path)
        cursor = conn.cursor()

        return conn, cursor

    else:
        return None, None

# conn, cursor = connect_to_sqlite(sqlite_db)

In [ ]:
def fetch_rows(cursor:sqlite3.Cursor,
                sql_query: str = """SELECT name FROM sqlite_master WHERE type='table';"""
                ) -> list:
    
    cursor = cursor.execute(sql_query)
    record = cursor.fetchall()
    return cursor, record


# _, table_names = fetch_rows(cursor)

In [ ]:
pipelines_q = """SELECT
                    pipelines.id AS id,
                    pipelines.origin AS pipeline
                FROM
                    pipelines
                """
# pipelines = pd.read_sql_query(pipelines_q, conn)

In [ ]:
avg_scores_q = """
                SELECT evaluation_id, AVG(value) AS avg_score
                FROM  evaluation_scores
                GROUP BY evaluation_id
                """


# scores = pd.read_sql_query(avg_scores_q, conn)
# avg_scores

In [ ]:
# pipelines = pipelines.join(avg_scores.set_index("evaluation_id"), on="evaluation_id")

In [ ]:
def make_pipeline_as_list(df:pd.DataFrame, col:str="pipeline") -> pd.DataFrame:
    def from_str_pipeline_to_list(p):
        # if "Template" in p:
        #     slice_start = 10
        # elif "AlphaAutoML" in p:
        #     slice_start = 13
        # p = p[slice_start:-1]



        p = p.strip("Template")
        p = p.strip("AlphaAutoML")
        p = p.strip()
        p = p.strip("()")
        p = p.split(",")
        return list(map(lambda x: "d3m.primitives." + x.strip(), p))
    df[col] = df[col].map(from_str_pipeline_to_list)
    return df

In [ ]:

# pipelines= make_pipeline_as_list(pipelines)

In [ ]:

def read_grammar_as_list(path: str) -> list:

    with open(path, "r") as infile:
        return infile.read().splitlines()


In [ ]:

# grammar_list = read_grammar_as_list(task_grammar)

In [ ]:
def get_component_from_p_rule(line:str) -> str:
    split_line = line.split(" ")
    component = split_line[-1]
    component = component.strip("'")
    if component == "E":
        return None
    return component





def get_component_type_lists_from_grammar(grammar:list) -> (list, list, list):


    # alphad3m/resource/primitives_hierarchy.json

    encoders_list_from_source_code = [
        "d3m.primitives.feature_extraction.count_vectorizer.SKlearn",
        "d3m.primitives.feature_extraction.tfidf_vectorizer.SKlearn",
        "d3m.primitives.data_transformation.encoder.DistilTextEncoder",
        "d3m.primitives.feature_construction.corex_text.DSBOX",
        "d3m.primitives.data_transformation.one_hot_encoder.SKlearn",
        "d3m.primitives.data_transformation.encoder.DSBOX",
        "d3m.primitives.data_transformation.ordinal_encoder.SKlearn",
        "d3m.primitives.data_transformation.label_decoder.Common",
        "d3m.primitives.data_transformation.label_encoder.Common",
        "d3m.primitives.data_cleaning.label_encoder.DSBOX",
        "d3m.primitives.data_transformation.enrich_dates.DistilEnrichDates",
        "d3m.primitives.data_transformation.encoder.DistilBinaryEncoder",
        "d3m.primitives.data_transformation.replace_singletons.DistilReplaceSingletons"]




    pipeline_template = None
    encoders=  encoders_list_from_source_code
    imputation = []
    feat_select = []
    feat_scale  =[]
    clfs = []

    imputation.append("d3m.primitives.data_cleaning.imputer.SKlearn")

    for line in grammar:

        if ("S ->" in line) and (not "ENCODERS ->" in line):
            template = line.split(" ")
            template = template[2:]# remove "S ->"
            template = list(map(lambda x: x.strip("'") , template))
            pipeline_template = template

        else:
            if ("ENCODERS" in line) or ("CATEGORICAL_ENCODER" in line):
                c = get_component_from_p_rule(line)
                if c != "CATEGORICAL_ENCODER":
                    encoders.append(c)
            elif "IMPUTATION" in line:
                c = get_component_from_p_rule(line)
                imputation.append(c)
            elif "FEATURE_SELECTION" in line:
                c = get_component_from_p_rule(line)
                feat_select.append(c)
            elif "FEATURE_SCALING" in line:
                c = get_component_from_p_rule(line)
                feat_scale.append(c)
            elif "CLASSIFICATION" in line:
                c = get_component_from_p_rule(line)
                clfs.append(c)
            else:
                print(f"Something is wrong? {line}")


    lambda_is_none = lambda x: x is not None

    encoders = list(filter(lambda_is_none, encoders))
    imputation = list(filter(lambda_is_none, imputation))
    feat_select = list(filter(lambda_is_none, feat_select))
    feat_scale = list(filter(lambda_is_none, feat_scale))


    return pipeline_template, encoders, imputation, feat_select, feat_scale, clfs


# pipeline_template, encoders, imputation, feat_select, feat_scale, clfs = get_component_type_lists_from_grammar(grammar_list)


In [ ]:

# pipelines["n_components"] = pipelines["pipeline"].map(lambda x: len(x))

In [ ]:
f_select_names = {
    "d3m.primitives.data_transformation.to_numeric.DSBOX":"ToNumeric (DSBOX)",
    "d3m.primitives.feature_selection.generic_univariate_select.SKlearn": "GenericUnivariateSelect",
    "d3m.primitives.feature_selection.select_percentile.SKlearn": "SelectPercentile",
    "d3m.primitives.feature_selection.variance_threshold.SKlearn" : "VarianceThreshold",
    "d3m.primitives.feature_selection.select_fwe.SKlearn" : "SelectFwe"}

f_scale_names = {
    "d3m.primitives.data_cleaning.max_abs_scaler.SKlearn": "MaxAbsScaler",
    "d3m.primitives.data_cleaning.min_max_scaler.SKlearn": "MinMaxScaler",
    "d3m.primitives.data_cleaning.quantile_transformer.SKlearn": "QuantileTransformer",
    "d3m.primitives.data_cleaning.robust_scaler.SKlearn": "RobustScaler",
    "d3m.primitives.data_cleaning.standard_scaler.SKlearn": "StandardScaler",
    "d3m.primitives.normalization.iqr_scaler.DSBOX": "IQRScaler (DSBOX)"
}

clfs_names ={
    "d3m.primitives.classification.ada_boost.SKlearn": "AdaBoostingClassifier",
    "d3m.primitives.classification.bagging.SKlearn": "BaggingClassifier",
    "d3m.primitives.classification.bernoulli_naive_bayes.SKlearn": "BernoulliNB",
    "d3m.primitives.classification.decision_tree.SKlearn" : "DecisionTreeClassifier",
    "d3m.primitives.classification.extra_trees.SKlearn": "ExtraTreesClassifier",
    "d3m.primitives.classification.gaussian_naive_bayes.SKlearn": "GaussianNB",
    "d3m.primitives.classification.gradient_boosting.SKlearn" : "GradientBoostingClassifier",
    "d3m.primitives.classification.k_neighbors.SKlearn": "KNeighborsClassifier",
    "d3m.primitives.classification.linear_discriminant_analysis.SKlearn": "LinearDiscriminantAnalysis",
    "d3m.primitives.classification.linear_svc.SKlearn": "LinearSVC",
    "d3m.primitives.classification.logistic_regression.SKlearn": "LogisticRegression",
    "d3m.primitives.classification.mlp.SKlearn": "MLPClassifier",
    "d3m.primitives.classification.multinomial_naive_bayes.SKlearn": "MultinomialNB",
    "d3m.primitives.classification.nearest_centroid.SKlearn" : "NearestCentroid",
    "d3m.primitives.classification.passive_aggressive.SKlearn": "PassiveAggressiveClassifier",
    "d3m.primitives.classification.quadratic_discriminant_analysis.SKlearn": "QuadraticDiscriminantAnalysis",
    "d3m.primitives.classification.random_forest.SKlearn": "RandomForestClassifier",
    "d3m.primitives.classification.sgd.SKlearn": "SGDClassifier",
    "d3m.primitives.classification.svc.SKlearn" : "SVC",
    "d3m.primitives.classification.Convolutional_neural_network.Fastai" : "ConvolutionalNN (FasAI)",
    "d3m.primitives.classification.light_gbm.Common": "LGBMClassifier (LightGBM)",
    "d3m.primitives.classification.xgboost_dart.Common" : "XGBoostClassifierDart (XGBoost)",
    "d3m.primitives.classification.xgboost_gbtree.Common" : "XGBoostClassifierGBTree (XGBoost)",
}

imputer_names = {
    "d3m.primitives.data_cleaning.imputer.SKlearn": "Imputer"
}

encoder_names = {
    "d3m.primitives.data_transformation.one_hot_encoder.SKlearn" : "OneHotEncoder",
    "d3m.primitives.data_transformation.encoder.DSBOX": "Encoder (DSBOX)",
    "d3m.primitives.feature_extraction.count_vectorizer.SKlearn": "CountVectorizer",
    "d3m.primitives.feature_extraction.tfidf_vectorizer.SKlearn": "TfidfVectorizer",
    "d3m.primitives.data_transformation.encoder.DistilTextEncoder": "TextEncoder (Distil)", # !!
    "d3m.primitives.feature_construction.corex_text.DSBOX" : "CorexText (DSBOX)",
    "d3m.primitives.data_transformation.ordinal_encoder.SKlearn": "OrdinalEncoder",
    "d3m.primitives.data_cleaning.label_encoder.DSBOX": "LabelEncoder (DSBOX)",
    "d3m.primitives.data_transformation.enrich_dates.DistilEnrichDates": "EnrichDates (Distil)",
    "d3m.primitives.data_transformation.replace_singletons.DistilReplaceSingletons": "ReplaceSingletons (Distil)",
    "d3m.primitives.data_transformation.label_encoder.Common": "LabelEncoder",
    "d3m.primitives.data_transformation.label_decoder.Common": "LabelDecoder",
    "d3m.primitives.data_transformation.encoder.DistilBinaryEncoder": "BinaryEncoder (Distil)"
}





names = [encoder_names, clfs_names, imputer_names, f_select_names, f_scale_names]





In [ ]:
def map_pipeline_c_names(pipeline):
    def mapper(component):
        for n in names:
            if n.get(component):
                component = n[component]
                break
        return component

    return list(map(mapper, pipeline))


# pipelines["pipeline"] = pipelines["pipeline"].map(map_pipeline_c_names)

In [ ]:
def mapper(component):
    for n in names:
        if n.get(component):
            component = n[component]
            break
    return component


In [ ]:


def get_component_types_and_counts_cols(df : pandas.DataFrame, types : dict) -> pandas.DataFrame:

    def count_components_of_type(row: pandas.Series, check_list:list) -> pd.Series:
        components_of_type = list(filter(lambda x: x in check_list, row["pipeline"]))
        return pd.Series([components_of_type])

    for k, v in types.items():
        df[k] = df.apply(count_components_of_type, args=(v,), axis = 1)
        df[f"n_{k}"] = df[k].map(len)

    return df


In [ ]:

# pipelines = get_component_types_and_counts_cols(pipelines)

In [ ]:

def make_edges(pipeline):
    edges = []
    edges.append(["Input Data", pipeline[0]])
    for ix in range(0, len(pipeline)-1):
        edges.append([pipeline[ix], pipeline[ix+1]])
    return edges




# pipelines["edges"] = pipelines["pipeline"].map(make_edges)

In [ ]:
def visualize(edges: list, out_name: str ,
              save_dir: str = None,
              graph_attrs: dict=None,
              view: bool = False,
              format="pdf") -> graphviz.Digraph:

    if graph_attrs is None:
        graph_attr= {'rankdir':'LR'}

    dot = graphviz.Digraph(out_name,
                      graph_attr=graph_attr)
    dot.format = format
    dot.edge_attr.update(arrowhead='vee', arrowsize='1.4')

    nodes = itertools.chain(*edges)
    for n in nodes:
        if "Input Data" in n:
            dot.node(n,  shape="cylinder", height="1.1")
        else:
            dot.node(n, height = "1.1")

    dot.edges(edges)


    dot.render(directory=save_dir, view=view)



# visualize(pipelines["edges"].iloc[0],"test",  os.path.join(kc1, "pipeline_vis"), format="png")

In [ ]:
d = "/home/hadi/PycharmProjects/Master-Thesis/zzz_automl_outputs/alphad3m-ds_splice-seed_662873"
temp = os.path.join(d, "temp")
sqlite_db = os.path.join(d, "db.sqlite3")
task_grammar = os.path.join(d, "task_grammar.bnf")

In [ ]:
sqlite_db

In [ ]:
conn, cursor = connect_to_sqlite(sqlite_db)


In [ ]:
conn

In [ ]:
pipelines = pd.read_sql_query(pipelines_q, conn)


In [ ]:
pipelines

In [ ]:
evals = pd.read_sql_query("SELECT * FROM evaluation_scores", conn)
evals

In [ ]:
conn.close()
pipelines= make_pipeline_as_list(pipelines)
grammar_list = read_grammar_as_list(task_grammar)
pipeline_template, encoders, imputation, feat_select, feat_scale, clfs = get_component_type_lists_from_grammar(grammar_list)

In [ ]:

encoders = list(map(mapper, encoders))
imputation = list(map(mapper, imputation))

feat_select = list(map(mapper, feat_select))

feat_scale = list(map(mapper, feat_scale))

clfs = list(map(mapper, clfs))


types = {
    "encoders": encoders,
    "imputation":imputation,
    "feat_select": feat_select,
    "feat_scale": feat_scale,
    "clfs": clfs
}


pipelines["n_components"] = pipelines["pipeline"].map(lambda x: len(x))
pipelines["pipeline"] = pipelines["pipeline"].map(map_pipeline_c_names)


pipelines = get_component_types_and_counts_cols(pipelines, types)
pipelines["edges"] = pipelines["pipeline"].map(make_edges)

In [ ]:
pipelines

In [ ]:
def get_summary_and_visualize(
    d_name:str,
    ROOT : str = "/home/hadi/PycharmProjects/Master-Thesis/automl_outputs/gpuserver/alphad3m") -> pandas.DataFrame :

    d = os.path.join(ROOT, d_name)
    temp = os.path.join(d, "temp")
    sqlite_db = os.path.join(temp, "db.sqlite3")
    task_grammar = os.path.join(temp, "task_grammar.bnf")




    conn, cursor = connect_to_sqlite(sqlite_db)
    pipelines = pd.read_sql_query(pipelines_q, conn)
    conn.close()
    pipelines= make_pipeline_as_list(pipelines)
    grammar_list = read_grammar_as_list(task_grammar)
    pipeline_template, encoders, imputation, feat_select, feat_scale, clfs = get_component_type_lists_from_grammar(grammar_list)

    encoders = list(map(mapper, encoders))
    imputation = list(map(mapper, imputation))

    feat_select = list(map(mapper, feat_select))

    feat_scale = list(map(mapper, feat_scale))

    clfs = list(map(mapper, clfs))


    types = {
        "encoders": encoders,
        "imputation":imputation,
        "feat_select": feat_select,
        "feat_scale": feat_scale,
        "clfs": clfs
    }


    pipelines["n_components"] = pipelines["pipeline"].map(lambda x: len(x))
    pipelines["pipeline"] = pipelines["pipeline"].map(map_pipeline_c_names)


    pipelines = get_component_types_and_counts_cols(pipelines, types)
    pipelines["edges"] = pipelines["pipeline"].map(make_edges)
    for ix, r in pipelines.iterrows():

        visualize(r["edges"], r["id"],  os.path.join(d, "pipeline_vis"), format="png")

    return pipelines


In [ ]:
kc1_pipelines = get_summary_and_visualize(kc1)

In [ ]:
kc1_pipelines["id"].duplicated().sum()

In [ ]:
kc1_pipelines.describe()
